# Calcium Imaging Video Viewer

## Summary

TODO:
- Short description and status

See this workflow's [readme](./readme_video-viewer.md) for more details about goals, specifications, and bottlenecks.

## Imports and Config

In [4]:
%%capture
%load_ext autoreload
%autoreload 2

from neurodatagen.imaging import simulate_miniscope_data
import panel as pn; pn.extension()
import hvplot.xarray
import holoviews as hv; hv.extension('bokeh')
from holoviews.streams import Stream
import param
from functools import partial
from dev.viz import VArrayViewer


# output_size = 100
# hv.output(size=output_size)

## Generate simulated data

The `simulate_miniscope_data` function generates synthetic calcium imaging data by simulating neural activity through spatial footprints and temporal calcium traces, while incorporating motion artifacts and realistic background noise. 

The function outputs a numpy array of 8-bit unsigned integers, encapsulated within an `xarray.DataArray`. This array represents the generated imaging data with dimensions corresponding to frame height, frame width, and frame number.

In [5]:
ncell = 15
dims = {'height': 600, 'width': 600, 'frame': 300}

data = simulate_miniscope_data(ncell=ncell, dims=dims)

In [6]:
data

<xarray.DataArray 'Simulated Miniscope Data' (frame: 300, height: 600,
                                              width: 600)>
dask.array<getitem, shape=(300, 600, 600), dtype=uint8, chunksize=(300, 600, 600), chunktype=numpy.ndarray>
Coordinates:
  * frame    (frame) int64 0 1 2 3 4 5 6 7 8 ... 292 293 294 295 296 297 298 299
  * height   (height) int64 0 1 2 3 4 5 6 7 ... 592 593 594 595 596 597 598 599
  * width    (width) int64 0 1 2 3 4 5 6 7 8 ... 592 593 594 595 596 597 598 599

## Plot Simulated Data

### Simple viewer using hvplot

In [7]:
# Wrapped in a Panel so that the widget stays close to the plot.. this should probably be the default for hvPlot
pn.panel(data.hvplot.image(groupby='frame', cmap='Viridis', frame_height=400, frame_width=400, colorbar=False))

Row
    [0] HoloViews(DynamicMap, sizing_mode='fixed')
    [1] WidgetBox(align=('end', 'start'))
        [0] DiscreteSlider(formatter='%d', margin=(20, 20, 20, 20), name='frame', options=OrderedDict([('0', ...]), value=0, width=250)

### Intermediate viewer using HoloViews DynamicMap of Image and a frame Stream linked to a Panel Player widget

In [8]:
frames = data.coords["frame"].values
f_min = int(frames.min())
f_max = int(frames.max())
height = data.sizes["height"]
width = data.sizes["width"]

# Generate Image object for a given frame
def generate_image(frame, data):  
    return hv.Image(data.sel(frame=frame).compute(), kdims=["width", "height"])

# Setup frame stream
frame_param = param.Integer(default=f_min, bounds=(f_min, f_max))
FrameStream = Stream.define("FrameStream", frame=frame_param)
frame_stream = FrameStream()

# Dynamic map of image via frame stream
image_generator = partial(generate_image, data=data)
image_map = hv.DynamicMap(image_generator, streams=[frame_stream]).opts(
    frame_width=500, aspect=width / height, cmap="Viridis"
)

# Create a video player widget
video_player = pn.widgets.Player(length=len(frames), interval=10, value=f_min, width=600, height=90)

# update the frame stream when a new event occurs on the widget
def update_frame_stream(event):
    frame_stream.event(frame=int(frames[event.new]))

# Link player widget to the frame stream update function
video_player.param.watch(update_frame_stream, "value")

# Layout

pn.layout.Column(video_player, image_map)

Column
    [0] Player(end=299, height=90, interval=10, sizing_mode='fixed', width=600)
    [1] HoloViews(DynamicMap, sizing_mode='fixed')

## Dev advanced viewer with color histogram, mask selection, summary stats, different input types

In [11]:
vaviewer = VArrayViewer(data)
vaviewer.show()

Column
    [0] WidgetBox
        [0] Button(button_type='primary', height=30, name='Update Mask', sizing_mode='fixed', width=100)
        [1] Player(end=299, height=90, interval=10, sizing_mode='fixed', width=650)
    [1] HoloViews(Layout)

## Load and plot real data